Toronto City Neighborhood Data
---
----


In [41]:
import pandas as pd
from bs4 import BeautifulSoup
import requests 

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
data = BeautifulSoup(source, 'lxml')

*importing modules 
define url and sourcing data in to "data" dataframe*

In [42]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

*columns named and data from columns into toronoto dataframe*

In [43]:
content = data.find('div', class_='mw-parser-output')

table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

*wrangled data from HTML table [td] & [tr] elements into dataframe columns named 'Postalcode' , 'Borough' , 'Neighborhood'*

In [44]:
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)

i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
                                 
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


*dropped 'Not assigned' Boroughs from table, converted 'Not Assigned' Locations with following location*

In [45]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode !=empty) & (df.Borough !=empty) & (df.Neighborhood !=empty)]

*drop "Not assigned" from dataframe*

In [46]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [47]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

print(df2.shape)
df2.head()

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


*Returned dimensions and header of dataframe*

In [48]:
!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
    
loc_df = pd.read_csv('Geospatial_Coordinates.csv')
loc_df.head()

--2020-02-22 18:39:27--  https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Resolving cocl.us (cocl.us)... 158.85.108.83, 169.48.113.194, 158.85.108.86
Connecting to cocl.us (cocl.us)|158.85.108.83|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-02-22 18:39:27--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197, 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-02-22 18:39:28--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/publ

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


*Pulled in csv data into pandas dataframe 'loc_df', reported the header for 'loc_df'* 

In [49]:
loc_df.rename(columns={'Postal Code':'Postalcode'}, inplace=True)

*renamed the 'Postal Code' column in loc_df in order to match the Geospatial_Coordinates data to the original df2 data where the column is named 'Postalcode'*

In [50]:
df3 = pd.merge(df2, loc_df, on = 'Postalcode', how = 'left')

*created a new data frame named 'df3' to hold all columns from df2 ('left' table) and the columns from loc_df (lat/long) which correspond to the same row in df2 based on data in column 'Postalcode'*

In [51]:
df3

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


*display full new dataframe with location data (df3)*


# Exploring Toronto Data

*enviromental setup, install libraries and modules*

In [30]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print('Environment Ready')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Environment Ready


In [52]:
import json

df3.to_json(path_or_buf='df3_toronto.json', orient='table')

In [53]:
print('Data downloaded!')

with open('df3_toronto.json') as json_data:
    toronto_df = json.load(json_data)

Data downloaded!


In [54]:
neigh_df = toronto_df['data']
neigh_df[0]

{'index': 0,
 'Postalcode': 'M1B',
 'Borough': 'Scarborough',
 'Neighborhood': 'Rouge, Malvern',
 'Latitude': 43.8066863,
 'Longitude': -79.1943534}

*create a dataframe for neighborhood data then order the columns below*

In [55]:
new_column_order = ['Postalcode','Borough','Neighborhood','Latitude','Longitude']

ndf = pd.DataFrame(neigh_df)
ndf = ndf[new_column_order]
ndf = ndf.sort_values(['Neighborhood', 'Latitude'], ascending=[True, True])
ndf.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
58,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
12,M1S,Scarborough,Agincourt,43.794200,-79.262029
14,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
89,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484


In [122]:
print('There are {} boroughs and {} neighborhoods.'.format(
            len(ndf['Borough'].unique()
               ),
            ndf.shape[0]
)
     )

There are 11 boroughs and 103 neighborhoods.


*how to get coordinates of a location in Toronto*

In [57]:
address = 'Alderwood'
geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinates of Alderwood are {}, {}.'.format(latitude, longitude))


Coordinates of Alderwood are 43.6017173, -79.5452325.


In [58]:
import folium


*To pull a random sample from df*

In [59]:
rand_ndf = ndf.sample(n=11)
toronto_map = folium.Map(location=[rand_ndf['Latitude'].mean(), rand_ndf['Longitude'].mean()], zoom_start=10)

for row in rand_ndf.itertuples():
   toronto_map.add_child(folium.Marker(location=[row.Latitude ,row.Longitude],
           popup=row.Borough))

*generate map*

In [60]:
toronto_map.save("toronto_map.html")

In [61]:
toronto_map


In [127]:
from folium.plugins import MarkerCluster
borough_map = folium.Map(location=[rand_ndf['Latitude'].mean(), 
 rand_ndf['Longitude'].mean()], 
 zoom_start=10)
mc = MarkerCluster()

for row in rand_ndf.itertuples():
    mc.add_child(folium.Marker(location=[row.Latitude,  row.Longitude],
                 popup=row.Borough))
    borough_map.add_child(mc)

borough_map.save("borough_map.html")
borough_map

*create a map for the boroughs, add markers to borough map and display map*

In [63]:
ndf.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
58,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
12,M1S,Scarborough,Agincourt,43.794200,-79.262029
14,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
89,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484


In [64]:
ndf.info

<bound method DataFrame.info of     Postalcode           Borough  \
58         M5H  Downtown Toronto   
12         M1S       Scarborough   
14         M1V       Scarborough   
101        M9V         Etobicoke   
89         M8W         Etobicoke   
28         M3H        North York   
19         M2K        North York   
62         M5M        North York   
56         M5E  Downtown Toronto   
9          M1N       Scarborough   
95         M9C         Etobicoke   
78         M6K      West Toronto   
87         M7Y      East Toronto   
30         M3K        North York   
68         M5V  Downtown Toronto   
51         M4X  Downtown Toronto   
74         M6E              York   
86         M7R       Mississauga   
4          M1H       Scarborough   
57         M5G  Downtown Toronto   
67         M5T  Downtown Toronto   
75         M6G  Downtown Toronto   
52         M4Y  Downtown Toronto   
7          M1L       Scarborough   
13         M1T       Scarborough   
8          M1M       Scarborough

In [65]:
print('There are {} boroughs and {} neighborhoods.'.format(
        len(ndf['Borough'].unique()),
        ndf.shape[0]
    )
)

There are 11 boroughs and 103 neighborhoods.


In [66]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinates of Toronto are {}, {}.'.format(latitude, longitude))

Coordinates of Toronto are 43.653963, -79.387207.


In [67]:
tn_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(ndf['Latitude'], ndf['Longitude'], ndf['Borough'], ndf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(tn_map)  

tn_map.save("tn_map.html")
tn_map

*creating the toronoto map with neighborhoods marked*

In [68]:
address = 'Downtown Toronto, Toronto'

geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinates of Downtown, Toronto are {}, {}.'.format(latitude, longitude))

Coordinates of Downtown, Toronto are 43.6541737, -79.38081164513409.


In [69]:
dtwn_data = ndf[ndf['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
dtwn_data

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
1,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
2,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420
3,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
5,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
8,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817
9,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576


In [70]:
dtwn_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 5 columns):
Postalcode      19 non-null object
Borough         19 non-null object
Neighborhood    19 non-null object
Latitude        19 non-null float64
Longitude       19 non-null float64
dtypes: float64(2), object(3)
memory usage: 840.0+ bytes


In [71]:
dtwn_toronto_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, label in zip(dtwn_data['Latitude'], dtwn_data['Longitude'], dtwn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(dtwn_toronto_map)  

dtwn_toronto_map.save("dtwn_toronto.html")
    
dtwn_toronto_map

*Downtown Toronto Neighborhood Map*

In [73]:
CLIENT_ID = 'U4KD030Y4D1W4KHOW2TIKATHHUT4Z1NS2XR1T5FHYHTMCMPZ'
CLIENT_SECRET = 'AZUKDNM2EGIRD1PPR5DVQY114QXGWTAJUTN00Q5ABI0TBFF0' 
VERSION = '20180604'
LIMIT = 30
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: U4KD030Y4D1W4KHOW2TIKATHHUT4Z1NS2XR1T5FHYHTMCMPZ
CLIENT_SECRET:AZUKDNM2EGIRD1PPR5DVQY114QXGWTAJUTN00Q5ABI0TBFF0


*Define Foursquare credentials and create GET function to retrieve data*

## Venues

*Clean columns and filter data for nearby venues*

In [161]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    dt_venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url2).json()["response"]['groups'][0]['items']
       
        dt_nearby_venues = json_normalize(results)
        dt_filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        dt_nearby_venues = dt_nearby_venues.loc[:, dt_filtered_columns]
        v=30
        dt_venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    dt_venues = pd.DataFrame(item for dt_nearby_venues in dt_venues_list for item in dt_nearby_venues)
    dt_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(dt_venues)

*Authenticate creditials for Foursquare, and create GET request for data*

In [181]:
dtwn_toronto = dtwn_data
toronto_venues = getNearbyVenues(names=dtwn_toronto['Neighborhood'],
                                   latitudes=dtwn_toronto['Latitude'],
                                   longitudes=dtwn_toronto['Longitude']
                                  )



Adelaide, King, Richmond
Berczy Park
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Cabbagetown, St. James Town
Central Bay Street
Chinatown, Grange Park, Kensington Market
Christie
Church and Wellesley
Commerce Court, Victoria Hotel
Design Exchange, Toronto Dominion Centre
First Canadian Place, Underground city
Harbord, University of Toronto
Harbourfront
Harbourfront East, Toronto Islands, Union Station
Queen's Park
Rosedale
Ryerson, Garden District
St. James Town
Stn A PO Boxes 25 The Esplanade


In [182]:
print(toronto_venues.shape)
toronto_venues.head()

(519, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Adelaide, King, Richmond",43.650571,-79.384568,Four Seasons Centre for the Performing Arts,43.650592,-79.385806,Concert Hall
1,"Adelaide, King, Richmond",43.650571,-79.384568,The Keg Steakhouse + Bar,43.649937,-79.384196,Steakhouse
2,"Adelaide, King, Richmond",43.650571,-79.384568,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant
3,"Adelaide, King, Richmond",43.650571,-79.384568,Nathan Phillips Square,43.652270,-79.383516,Plaza
4,"Adelaide, King, Richmond",43.650571,-79.384568,Shangri-La Toronto,43.649129,-79.386557,Hotel


*Display data for local venues in the Downtown Toronto area,
-- then count venues in each area*

In [185]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",30,30,30,30,30,30
Berczy Park,30,30,30,30,30,30
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",17,17,17,17,17,17
"Cabbagetown, St. James Town",30,30,30,30,30,30
Central Bay Street,30,30,30,30,30,30
"Chinatown, Grange Park, Kensington Market",30,30,30,30,30,30
Christie,18,18,18,18,18,18
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30


In [186]:
print('{} unique categories'.format(len(dtwn_toronto_venues['Venue Category'].unique())))

149 unique categories


In [188]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()


,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Liquor Store,Lounge,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skating Rink,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Adelaide, King, Richmond",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Adelaide, King, Richmond",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Adelaide, King, Richmond",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Adelaide, King, Richmond",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Adelaide, King, Richmond",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [189]:
toronto_onehot.shape


(519, 149)

*Group data by neighborhood and by the average of number of locations* 

In [190]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Liquor Store,Lounge,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skating Rink,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.066667,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.033333,0.066667,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.033333,0.000000,0.000000,0.033333,0.0333

In [191]:
toronto_grouped.shape

(19, 149)

*create a dataframe with average counts showing frequency of venues*

In [192]:
num_top_venues = 5
for neigh in toronto_grouped['Neighborhood']:
    print("----"+neigh+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0              Café  0.10
1             Hotel  0.07
2  Asian Restaurant  0.07
3  Sushi Restaurant  0.07
4        Steakhouse  0.07


----Berczy Park----
            venue  freq
0     Coffee Shop  0.10
1            Café  0.07
2          Bakery  0.07
3  Farmers Market  0.07
4    Cocktail Bar  0.07


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3       Coffee Shop  0.06
4           Airport  0.06


----Cabbagetown, St. James Town----
                venue  freq
0                Café  0.07
1              Bakery  0.07
2          Restaurant  0.07
3  Italian Restaurant  0.07
4         Coffee Shop  0.07


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.23
1   Italian Restaurant  0.07
2  Japanese Restaurant  0.07
3   Chinese Restauran

In [194]:
def return_freq(row, num_top_venues):
    row_cat = row.iloc[1:]
    row_cat_sort = row_cat.sort_values(ascending=False)
    return row_cat_sort.index.values[0:num_top_venues]

In [199]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_freq(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(19, 11)

In [201]:
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)

print(kmeans.labels_[0:10])
print(len(kmeans.labels_))

[7 2 3 5 9 4 6 0 5 5]
19


In [202]:
dtwn_toronto.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
1,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
2,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420
3,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [205]:
toronto_merged = dtwn_toronto
toronto_merged['Cluster Labels'] = kmeans.labels_
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head() 

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,7,Café,Hotel,Asian Restaurant,Sushi Restaurant,Steakhouse,Greek Restaurant,Speakeasy,Noodle House,Opera House,Colombian Restaurant
1,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Cocktail Bar,Café,Beer Bar,Bakery,Farmers Market,Steakhouse,Museum,Park,Concert Hall
2,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,3,Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Boat or Ferry,Sculpture Garden,Rental Car Location,Plane,Boutique
3,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,5,Italian Restaurant,Bakery,Café,Restaurant,Coffee Shop,Gift Shop,Diner,Market,Park,Caribbean Restaurant
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,9,Coffee Shop,Italian Restaurant,Chinese Restaurant,Japanese Restaurant,Poke Place,Seafood Restaurant,Sandwich Place,Ramen Restaurant,Comic Shop,Portuguese Restaurant


## Visualize Data with the top venues in the  
## Downtown Toronto Neighborhoods

In [213]:
venue_cat_map = folium.Map(location=[latitude, longitude], zoom_start=13)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'],kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(venue_cat_map)
       
venue_cat_map